In [61]:
from ollama import  generate
import gradio as gr
import time
import re
import os

In [30]:
def get_llm_response(input_text, task='revision'):
    """
    Parameters:
        model_name: str
            Name of the model to be used for the task
        input_text: str
            Input text from whisper transcribed result
        task: str
            Task to be performed by the model. 'revision' means to correct the input text but do not change the content.
            'response' means to generate a response to the input text.
    Returns:
        json format file for 'ie' task. 
        conversation text format file for 'ca' task.
    """    
    revision_prompt = """
    あなたは法律文書の専門家です。以下の指示に従って文書を処理してください：

    基本方針：
    - 原文をできるだけ尊重する
    - 明らかな誤りのみを修正する
    - 法的に問題のある箇所のみを修正する
    - 文体や表現の好みによる変更は避ける
    - 修正理由や説明は省略する

    修正対象：
    - 法律用語の明らかな誤用
    - 法令の引用の誤り
    - 重大な文法的誤り
    - 明らかな事実誤認
    - 法的な論理の矛盾

    出力形式：
    [修正後の文書のみを出力]
    """
    response_prompt = "あなたは専門的な日本法律顧問です。質問されたことに対して専門的な法律の見地から日本語で答えてください。重複内容を出力しないでください"

    model_name = 'llama3.3_jp_keiyaku_1221_Q4KM'

    generation_params = {
        #"do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
        "top_k": 40,
        "num_predict": 1024,
        "num_ctx": 2048,
        "repeat_penalty": 1.2,
    }

    if task == 'revision':

        system_prompt = revision_prompt

        res = generate(
            model=model_name,
            prompt=input_text,
            system=system_prompt,
            options=generation_params
        )
    elif task == 'response':

        system_prompt = response_prompt

        res = generate(
            model=model_name,
            prompt=input_text,
            system=system_prompt,
            options=generation_params
        )
    else:
        print("Invalid task. Please choose either 'revision' or 'response'.")

    response = res['response']

    current_dir = os.path.dirname(os.path.realpath(__file__))
    data_path = current_dir + '/data'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    date_today = time.strftime("%Y%m%d")
    output_file = data_path + '/record_' + date_today + '.txt'
    with open(output_file, 'a', encoding='utf-8-sig') as f:
        f.write(f"Input text: {input_text}\n")
        f.write(f"Response: {response}\n")

    for i in range(len(response)):
        yield response[:i+1]
        time.sleep(0.01)

    # return res['response']

In [18]:
res = get_llm_response("売主（以下「甲」という）と買主（以下「乙」という）とは、次の通り合意した。 第×条…（例1） 甲は乙に対し、目的物の代金を支払う。", task='revision')
print(res)

売主（以下「甲」という）と買主（以下「乙」という）とは、次の通り合意した。 第×条…（例1） 乙は甲に対し、目的物の代金を支払う。


In [32]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # タイトル
    gr.Markdown("# 契約書内容処理アプリ")
    gr.Markdown("このアプリでは、契約書内容の修正や質問への回答が可能です。")
    
    # 入力テキストエリア
    with gr.Row():
        input_text = gr.Textbox(
            label="契約書内容を入力してください",
            placeholder="ここに処理したいテキストを入力してください...",
            lines=10
        )
    
    # タスク選択エリア
    with gr.Row():
        task_radio = gr.Radio(
            choices=[("回答する", "response"), ("修正する", "revision")],
            label="タスクを選択してください",
            value="response"
        )
    
    # 送信ボタン
    with gr.Row():
        submit_btn = gr.Button("送信", variant="primary")
    
    # 出力エリア
    with gr.Row():
        output_text = gr.Textbox(
            label="処理結果",
            lines=20
        )
    
    # クリックイベントの設定
    submit_btn.click(
        fn=get_llm_response,
        inputs=[input_text, task_radio],
        outputs=output_text,
        api_name="stream_output"
    )


In [35]:
demo.queue()
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [34]:
demo.close()

Closing server running on port: 7860


In [55]:
def detect_and_process_numbered_list(text):
    """
    检测和处理带编号的列表部分
    保留其他文本内容
    """
    # 分行处理
    lines = text.split('\n')
    # 存储处理后的行
    processed_lines = []
    # 存储已见过的内容
    seen_contents = {}
    # 当前编号计数
    current_number = 1
    
    # 用于检测编号行的正则表达式
    # 匹配格式如: "1.", "1。", "1、", "(1)", "１．"等
    number_pattern = r'^[\(（]?\d+[\)）\.\。、．]'
    
    in_numbered_list = False  # 标记是否在处理编号列表
    for line in lines:
        stripped_line = line.strip()
        if not stripped_line:  # 保留空行
            processed_lines.append(line)
            continue
            
        # 检查是否是编号行
        if re.match(number_pattern, stripped_line):
            in_numbered_list = True
            # 提取内容部分（去除编号）
            content = re.sub(number_pattern, '', stripped_line).strip()
            
            # 如果内容不重复，添加到结果中
            if content not in seen_contents:
                seen_contents[content] = True
                # 使用当前编号重新格式化行
                processed_lines.append(f"{current_number}. {content}")
                current_number += 1
        else:
            # 如果已经在处理编号列表，且遇到非编号行，说明列表结束
            if in_numbered_list:
                in_numbered_list = False
            # 非编号行直接添加
            processed_lines.append(line)
    
    return '\n'.join(processed_lines)


In [59]:
test_text ="""
lsdjfldjlskfljksdjflk21234214123
jsdfjkl3912321
1.売主及び買主に関する記述
売主及び買主に関する記述
2.本契約の目的及び取引の対象となる株式
3.売主の表明及び保証
4.買主の表明及び保証
5.売主の義務及び責任
6.買主の義務及び責任
7. 買主の義務及び責任
7.クロージングに関する事項
8.契約締結後における売主の義務
9.本契約に定める違反についての損害賠償責任
10.本契約に定める違反についての損害賠償責任
11.本契約に定める違反についての損害賠償責任
12.本契約に定める違反についての損害賠償責任
jslkdfjl29jlksdlkfnc,mzcdsfds2.f
"""

In [60]:
print(detect_and_process_numbered_list(test_text))


lsdjfldjlskfljksdjflk21234214123
jsdfjkl3912321
1. 売主及び買主に関する記述
売主及び買主に関する記述
2. 本契約の目的及び取引の対象となる株式
3. 売主の表明及び保証
4. 買主の表明及び保証
5. 売主の義務及び責任
6. 買主の義務及び責任
7. クロージングに関する事項
8. 契約締結後における売主の義務
9. 本契約に定める違反についての損害賠償責任
jslkdfjl29jlksdlkfnc,mzcdsfds2.f

